# Numerično integriranje

In [1]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
%matplotlib inline

/opt/anaconda/lib/python3.6/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


### vprašanje 1 (wan) - sympy

In [8]:
R = 7.5
r, p = sym.symbols('r, varphi')
f_sym = r**3 / 3 *sym.sin(p)
I_sym = sym.integrate(f_sym, (p, 0, sym.pi/2))
I_pravi = I_sym.subs(r, R).evalf(10) #brez evalf ni pravi float - ne moremo računat z njim čeprav je že štefilka
I_pravi

140.6250000

In [11]:
f_num = sym.lambdify(p, f_sym.subs(r, R), 'numpy') #numpyeva funkcija
f_num?

### vprašanje 2

In [13]:
x, h = np.linspace(0, np.pi/2, 100, retstep=True)
y = f_num(x)

In [14]:
from scipy.integrate import simps

In [16]:
I_trapz = np.trapz(y, x) #sestavljena trapezna metoda
I_trapz

140.62204979308572

In [21]:
I_simps = simps(y, x) #natančnejša #simps(y, dx=h)
I_simps

140.62497645950683

### Sestavljeno trapezno pravilo (wan)

In [23]:
x, h = np.linspace(0, np.pi/2, 100, retstep=True)
y = f_num(x)
(np.sum(y) - (y[0]/2 + y[-1]/2)) * (x[1] - x[0]) #lažje odštet polovičke od vsega * h

140.62204979308575

### vprašanje 3

In [24]:
#z uporabo osnovnega trapeznega pravila (ne sestavljenega)
def f(x):
    return -0.5*x**3 - x**2 + 8

In [36]:
x = np.array([0, 2]) #za osnovno trapezno rabimo samo 2 točki - prvo in zadnjo (meje)
y = f(x)
I_osnovno = (np.sum(y)) * (x[1] - x[0])/2
I_osnovno

8.0

In [38]:
x_11, h_11 = np.linspace(0, 2, 11, retstep=True)
y_11 = f(x_11)
I_sestavljeno = np.trapz(y_11, dx=h_11)
I_sestavljeno

11.300000000000001

### vprašanje 4 - izboljšan približek z Richardsovo ekstrapolacijo
* izračunamo z korakom h in 2h
* boljši približek
* trapezno - red napake 2 => n=2

In [39]:
x_1 = np.linspace(0, 2, 3)
x_2 = np.linspace(0, 2, 5)
y_1 = f(x_1)
y_2 = f(x_2)
I_h = np.trapz(y_2, x_2)
I_2h = np.trapz(y_1, x_1) #večji korak - manj točk
I_boljši = 4/3 * I_h - 1/3 * I_2h #boljšega bolj upoštevamo - večja utež
I_boljši
# f = polinom 3 red - richardsova ekstrapolacija točna vrednost polinoma 3. reda z 5 točkami! ne rabimo več

11.333333333333332

### vprašanje 6 - Simpsonovo 1/3 pravilo
* A_i = h * [1/3, 4/3, 1/3] ---> uteži (za 3 točke)
* A_i = h * [1/3, 4/3, 2/3, 4/3, 1/3] ---> uteži (za 5 točk)

In [49]:
utezi = np.array([1/3, 4/3, 1/3])
x1 = np.array([0, 1, 2]) #3 točke med 0 in 2
y1 = f(x1)
I_simps_osnovna = np.dot(utezi, y1) * 1
I_simps_osnovna #z simpsoms smo sobili z 3mi točkami pravo vrednost - pri richardsu rsmo rabili 5 točk
#simpsonovo pravilo ima napako 4 reda - polinome 3. reda in manj pravilno izračuna!
#simps = interpolacija 2 reda. - pri 3.redu je napaka simetrična - tolko ku smo nikje zračunali prevč smo na drugmi mesti zračunali premalo.

11.333333333333332

In [42]:
from scipy.integrate import simps

In [52]:
x2 = np.linspace(0, 2, 7)
y2 = f(x2)
I_simps = simps(y2, x2)
I_simps

11.333333333333332

### Simpsonovo 3/8 pravilo

In [53]:
from scipy.integrate import newton_cotes

In [55]:
newton_cotes(1)[0] #trapezno - linearna

array([ 0.5,  0.5])

In [56]:
newton_cotes(2)[0] #simpsonovo 1/3 - kvadratna

array([ 0.33333333,  1.33333333,  0.33333333])

In [57]:
newton_cotes(3)[0] #simpsonovo 3/8 - polinom 3 reda

array([ 0.375,  1.125,  1.125,  0.375])

In [64]:
red = 6
utezi = newton_cotes(red)[0]
x_4, h = np.linspace(0, 2, red+1, retstep = True)
integral = utezi @ f(x_4) * h
integral

11.333333333333332

### Gaussova integracijska metoda
* quad(fun, a, b) -podamo funkcijo 
* fixed_quad(fun, a, b, n) - podamo funkcijo
* ... en kp funkcij
* moramo podati funkcijo ker gaussova integracija potrebuje točno določene vrednosti, ki jih z meritvami ne nujno dobimo

### vprašanje 8 (wan)

In [65]:
from scipy.integrate import quad, fixed_quad

In [68]:
def f(x):
    return np.sin(x)/x #vrednost te funkcije v nuli neznamo zračunat - pol -> trapezna metoda ne zna zračunat!
#gaussova ne rabi v mejah - samo določene točke, v katerih ne sme bit pol (zelo redko po naključju)
#gaussa ne motijo poli

In [71]:
I_1 = quad(f, 0, 1)
I_1[0] #druga vrednost je ocena napake

0.9460830703671831

In [72]:
I_1 = quad(f, 0, 2)
I_1[0]

1.6054129768026946

In [73]:
I_1 = quad(f, -0.5, 0.5)
I_1[0] #vrne not a number
#v polu želi izračunati točko gauss

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/anaconda/lib/python3.6/site-packages/scipy/integrate/quadpack.py:364: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


nan

In [74]:
#z trapezno
x = np.linspace(0, 1, 50)
np.trapz(f(x), x)
#v nuli ne zna zračunat

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


nan

In [78]:
#iz pastebina
#kako določit uteži gaussove integracije
from scipy.special import legendre
red = 3
xi = legendre(red).weights[:,0]
Ai = legendre(red).weights[:,1]
print(xi)
print(Ai)

[-0.77459667  0.          0.77459667]
[ 0.55555556  0.88888889  0.55555556]


In [85]:
#Gauss ročno:

In [82]:
#kako dobiti x na območju -1,1
a = 1
b = 2
x = xi*(b-a)/2 + (b+a)/2
y = f(x)
I = np.dot(Ai, y) * (b-a)/2 # (b-a)/2 -> pretvorimo na naš interval a,b
I

0.65932992413122304

In [84]:
fixed_quad(f, a, b, n=3)[0]

0.65932992413122304